In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='YSwhyVuDKf8kHO0pQVPIdTs-3K5TQRYn3cY35rpl9s-F',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

bucket = 'nalaiyathiran-donotdelete-pr-isnmlnmxzneupt'
object_key = 'dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
from io import BytesIO
import zipfile

zip_file = zipfile.ZipFile(BytesIO(streaming_body_1.read()), 'r')
zip_file.extractall(".")
zip_file.close()

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [10]:
train_data_gen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_data_gen = ImageDataGenerator(rescale=1./255)
train_data = train_data_gen.flow_from_directory(directory="data/train", target_size=(64, 64), color_mode='grayscale', class_mode='categorical', batch_size=32)
test_data = test_data_gen.flow_from_directory(directory="data/test", target_size=(64, 64), color_mode='grayscale', class_mode='categorical', batch_size=32)

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [13]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D

In [19]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(32))
model.add(Dense(6, activation="softmax"))

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 32)               

In [21]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [22]:
model.fit(train_data, epochs=10, validation_data=test_data)

Epoch 1/10
480/480 [==============================] - 35s 73ms/step - loss: 0.7992 - accuracy: 0.7329 - val_loss: 0.5883 - val_accuracy: 0.8299
Epoch 2/10
480/480 [==============================] - 34s 71ms/step - loss: 0.3199 - accuracy: 0.9046 - val_loss: 0.4432 - val_accuracy: 0.8633
Epoch 3/10
480/480 [==============================] - 32s 67ms/step - loss: 0.2583 - accuracy: 0.9247 - val_loss: 0.5016 - val_accuracy: 0.8552
Epoch 4/10
480/480 [==============================] - 35s 72ms/step - loss: 0.2231 - accuracy: 0.9350 - val_loss: 0.3988 - val_accuracy: 0.8828
Epoch 5/10
480/480 [==============================] - 33s 69ms/step - loss: 0.1886 - accuracy: 0.9439 - val_loss: 0.3399 - val_accuracy: 0.8968
Epoch 6/10
480/480 [==============================] - 34s 70ms/step - loss: 0.1720 - accuracy: 0.9494 - val_loss: 0.2473 - val_accuracy: 0.9253
Epoch 7/10
480/480 [==============================] - 34s 70ms/step - loss: 0.1545 - accuracy: 0.9530 - val_loss: 0.2707 - val_accuracy:

In [36]:
from tensorflow.keras.preprocessing import image
image_to_predict = np.expand_dims(image.img_to_array(
            image.load_img('data/test/Left Bundle Branch Block/fig_5897.png', target_size=(64, 64), color_mode="grayscale")), axis=0)

In [40]:
CVDs = [
            'left_bundle_branch_block',
            'normal',
            'premature_atrial_contraction',
            'premature_ventricular_contractions',
            'right_bundle_branch_block',
            'ventricular_fibrillation'
        ]

In [41]:
prediction = model.predict(image_to_predict)
CVDs[list(prediction[0]).index(1)]

'left_bundle_branch_block'

In [42]:
model.save('model_latest.h5')